In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt 
from matplotlib import rcParams 
rcParams['figure.figsize'] = 8,6
import seaborn as sb
sb.set()

In [ ]:
btc = pd.DataFrame(pdr.get_data_yahoo('BTC-USD')['Close'])
btc.head()

In [ ]:
ma = 21
btc['return'] = np.log(btc["Close"]).diff()
btc['ma'] = btc['Close'].rolling(ma).mean()
btc['ratio'] = btc['Close'] / btc['ma']
btc

In [ ]:
btc['ratio'].describe()

In [ ]:
percentiles = [2.5, 7, 50, 95, 97.5]
p = np.percentile(btc['ratio'].dropna(), percentiles)

In [ ]:
btc['ratio'].dropna().plot(legend =True)
plt.axhline(p[0], c= (.5, .5, .5), ls='--')
plt.axhline(p[2], c= (.5, .5, .5), ls='--')
plt.axhline(p[-1], c= (.5, .5, .5), ls='--')


In [ ]:
short = p[-1]
long = p[0]
btc['position'] = np.where(btc.ratio > short, -1, np.nan)
btc['position'] = np.where(btc.ratio < long, 1, btc['position'])
btc['position'] = btc['position'].ffill()


In [ ]:
btc.position.dropna().plot()

In [ ]:
btc['strat_return'] = btc['return'] * btc['position'].shift()

In [ ]:
plt.plot(np.exp(btc['return'].dropna()).cumprod(), label='Buy/Hold')
plt.plot(np.exp(btc['strat_return'].dropna()).cumprod(), label='Strategy')
plt.legend();